In [1]:
from os import path
import requests, sys, webbrowser, bs4
import pymongo
import pickle
import random

end_url = '_(California)'

errors = ['Kevin_Cavanaugh_(New_Hampshire)']

state_gov_nm = 'CA'
state_gov_id = 5

data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

_sponsor_comittee_table = path.join(data_file_location, state_gov_nm + 'sponsor_comittee_table.dict')


mongo_username= 'CEDC_Quigley'
mongo_password= 'lpP9pCcDxQ2Y0tv8'
mongo_server = 'mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
mongo_authSource = 'admin'
mongo_authMechanism = 'SCRAM-SHA-1'
mongo_database = 'legislative_documents'
mongo_people_collection = 'legiscan_people_data'

client = pymongo.MongoClient(mongo_server
                        ,username=mongo_username
                        ,password=mongo_password
                        ,authSource=mongo_authSource
                        ,authMechanism=mongo_authMechanism)



In [2]:

database = client[mongo_database]
collection = database[mongo_people_collection]


positions = ["vice chair", "clerk", "chair"] # other position is member
root = "https://ballotpedia.org/"
committee_table = {}
politicians = []
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')


for people in collection.find({'state_id':state_gov_id}):

    try:

        p = people['ballotpedia']

        #end_url = "_(New_Hampshire)"

        committee_table[p] = {'year':[], 'committee/positions': []}


        url = root + p
        res = requests.get(url)
        res.raise_for_status()

        soup = bs4.BeautifulSoup(res.text, features="html.parser")
        check_multi_name = soup.find('table', id="disambig")

        if not check_multi_name is None:
            new_url = url + end_url
            new_res = requests.get(new_url)
            new_res.raise_for_status()

            soupy = bs4.BeautifulSoup(new_res.text, features="html.parser")


            sessions = soupy.find_all('table')

            for j in range(len(sessions)):

                val = sessions[j].find('b')
                try:
                    stringy = val.text
                    res = stringy.lower()
                    if "committee" in res:
                        _time = int(res[-4:])
                        if _time < 2015:
                            temp_val = _time + 1
                            time = str(_time) + "-" + str(temp_val)
                        else:
                            time = str(_time)

                        tempy = sessions[j].find_all('td')
                        for t in tempy:
                            _ret = ''.join(filter(whitelist.__contains__, t.text))
                            ret = _ret.lower()

                            committee_table[p]['year'].append(time)
                            insert_mask = 0
                            for pos in positions:
                                if pos in ret:
                                    committee = ret.replace(pos, "")
                                    result = committee + "/" + pos
                                    committee_table[p]['committee/positions'].append(result)
                                    insert_mask = 1
                                    break
                            if insert_mask == 0:
                                committee = ret
                                result = committee + "/" + "member"
                                committee_table[p]['committee/positions'].append(result)

                except:
                    pass

        else:
            sessions = soup.find_all('table')

            for j in range(len(sessions)):

                val = sessions[j].find('b')
                try:
                    stringy = val.text
                    res = stringy.lower()
                    if "committee" in res:

                        _time = int(res[-4:])

                        if _time < 2015:
                            temp_val = _time + 1
                            time = str(_time) + "-" + str(temp_val)
                        else:
                            time = str(_time)
                        tempy = sessions[j].find_all('td')
                        for t in tempy:
                            _ret = ''.join(filter(whitelist.__contains__, t.text))
                            ret = _ret.lower()

                            committee_table[p]['year'].append(time)
                            insert_mask = 0
                            for pos in positions:
                                if pos in ret:
                                    committee = ret.replace(pos, "")
                                    result = committee + "/" + pos
                                    committee_table[p]['committee/positions'].append(result)
                                    insert_mask = 1
                                    break
                            if insert_mask == 0:
                                committee = ret
                                result = committee + "/" + "member"
                                committee_table[p]['committee/positions'].append(result)

                except:
                    pass
    except:
        pass

In [3]:
with open(_sponsor_comittee_table, 'wb') as f:
    pickle.dump(committee_table, f)